### Setup colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/colab-projects/EEG_AD_Detection/

/content/drive/MyDrive/colab-projects/EEG_AD_Detection


In [ ]:
# !mkdir -p ~/.ssh
# !chmod 700 ~/.ssh
# !ssh-keyscan github.com >> ~/.ssh/known_hosts
# !chmod 600 ~/.ssh/known_hosts
# !cp "/content/drive/MyDrive/ssh-keys/id_rsa" ~/.ssh/id_rsa
# !chmod 400 ~/.ssh/id_rsa
# import os
# os.environ['GIT_SSH_COMMAND'] = 'ssh -i ~/.ssh/id_rsa'
# # !git clone git@github.com:VThuong99/EEG_AD_Detection.git '/content/drive/MyDrive/colab-projects/EEG_AD_Detection'


In [ ]:
# !git add config.py
# !git commit -m "Change config.py"

In [ ]:
# !git pull

In [ ]:
# !git config --global user.email "duongvanthuong9a8@gmail.com"
# !git config --global user.name "VThuong99"

In [3]:
# !pip install -r requirements.txt
!pip install mne
!pip install mne_connectivity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.9 MB/s eta 0:00:00


In [4]:
%load_ext autoreload
%autoreload 2

### Data preprocessing (Split into epochs)

In [5]:
from src.data_processing.load_data import *

In [6]:
# process_data(duration=4, overlap=2, classes= {'A': 1, 'C': 0})
# process_data(duration=4, overlap=0, classes= {'A': 1, 'C': 0})

In [7]:
subject_datas, targets = load_processed_data(duration=4, overlap=2, processed_path=PROCESSED_DATA_PATH)

In [8]:
print(f"subject data type: {type(subject_datas)}")
shapes = []
for subject in subject_datas:
    shapes.append(subject.shape)
print(shapes)
print(f"targets type: {type(targets)}")
print(f"targets: {targets}")

subject data type: <class 'list'>
[(298, 19, 2000), (395, 19, 2000), (152, 19, 2000), (352, 19, 2000), (401, 19, 2000), (315, 19, 2000), (382, 19, 2000), (395, 19, 2000), (305, 19, 2000), (639, 19, 2000), (383, 19, 2000), (440, 19, 2000), (418, 19, 2000), (466, 19, 2000), (449, 19, 2000), (485, 19, 2000), (420, 19, 2000), (421, 19, 2000), (458, 19, 2000), (433, 19, 2000), (459, 19, 2000), (409, 19, 2000), (416, 19, 2000), (378, 19, 2000), (341, 19, 2000), (448, 19, 2000), (412, 19, 2000), (408, 19, 2000), (367, 19, 2000), (276, 19, 2000), (574, 19, 2000), (397, 19, 2000), (351, 19, 2000), (483, 19, 2000), (369, 19, 2000), (419, 19, 2000), (387, 19, 2000), (444, 19, 2000), (424, 19, 2000), (481, 19, 2000), (442, 19, 2000), (480, 19, 2000), (413, 19, 2000), (439, 19, 2000), (424, 19, 2000), (375, 19, 2000), (402, 19, 2000), (493, 19, 2000), (389, 19, 2000), (407, 19, 2000), (375, 19, 2000), (379, 19, 2000), (389, 19, 2000), (417, 19, 2000), (406, 19, 2000), (394, 19, 2000), (397, 19, 200

### Feature extraction

In [11]:
from src.data_processing.feature_extraction import PsdFeature, RbpFeature, SccFeature, MeanFeature, VarianceFeature, FlattenFeature, FeaturePipeline

In [12]:
feature_extractors = [RbpFeature()]

In [13]:
pipeline = FeaturePipeline(feature_extractors)

In [14]:
features = []
for subject in subject_datas:
  feature = pipeline.run(subject, sfreq=500)
  features.append(feature)

print(f"features type: {type(features)}")
print(f"features shape: {features[0].shape}")

features type: <class 'list'>
features shape: (298, 19, 5)


### Test full pipeline

In [ ]:
len(targets)

65

In [ ]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from src.validation.cross_validation import LOSOCV  # Import the LOSOCV class

# 1. Process data (Sliding window + Feature extraction)
# 2. Define a KNN Model (you can replace this with any model that has fit/predict methods)
knn_model = KNeighborsClassifier(n_neighbors=3)

# 3. Run the Model
loocv = LOSOCV(knn_model) # Run all of the data
train_metrics, test_metrics = loocv.run(features, targets)

print(f"Train Metrics: {train_metrics}")
print(f"Test Metrics: {test_metrics}")

#Optional: Run the data on smaller sample size
n_folds = 3 # Set the number of folds
random_state = 42 # Set some number for the random_state
loocv_partial = LOSOCV(knn_model, metrics=['accuracy', 'f1'], n_folds = n_folds, random_state = random_state)  # Run all of the data with metrics limited to the  accuracy and f1
train_metrics, test_metrics = loocv_partial.run(features, targets) # Now get training metrics

print(f"Test Partial Data: {test_metrics}")

Train Metrics: {'accuracy': 0.9195897514107061, 'sensitivity': 0.9302194710185706, 'specificity': 0.907009657009657, 'f1': 0.9261477045908183}
Test Metrics: {'accuracy': 0.6577181208053692, 'sensitivity': 0.6577181208053692, 'specificity': 0, 'f1': 0.7935222672064777}
Test Partial Data: {'accuracy': 0.26634382566585957, 'f1': 0}


### Knn with Rbp and mean

In [15]:
from sklearn.neighbors import KNeighborsClassifier

from src.validation.cross_validation import LOSOCV  # Import the LOSOCV class

# 1. Process data (Sliding window + Feature extraction)
feature_extractors = [RbpFeature()]
pipeline = FeaturePipeline(feature_extractors)
features = []
for subject in subject_datas:
  feature = pipeline.run(subject, sfreq=500)
  features.append(feature)

# 2. Define a KNN Model (you can replace this with any model that has fit/predict methods)
knn_model = KNeighborsClassifier(n_neighbors=3)

# 3. Run the Model
loocv = LOSOCV(knn_model) # Run all of the data
train_metrics, test_metrics = loocv.run(features, targets)

print(f"Train Metrics: {train_metrics}")
print(f"Test Metrics: {test_metrics}")

Train Metrics: {'accuracy': 0.9195897514107061, 'sensitivity': 0.9302194710185706, 'specificity': 0.907009657009657, 'f1': 0.9261477045908183}
Test Metrics: {'accuracy': 0.6577181208053692, 'sensitivity': 0.6577181208053692, 'specificity': 0, 'f1': 0.7935222672064777}


#### Experiment with some traditional ML models

In [16]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, Trials
from src.validation.cross_validation import LOSOCV  # Import the LOSOCV class

# 1. Process data (Sliding window + Feature extraction)
feature_extractors = [RbpFeature()]
pipeline = FeaturePipeline(feature_extractors)
features = []
for subject in subject_datas:
    feature = pipeline.run(subject, sfreq=500)
    features.append(feature)

# Define models
models = {
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM (Poly Kernel)": SVC(kernel='poly', degree=3, probability=True),
    "MLP (1 Hidden Layer, 3 Neurons)": MLPClassifier(hidden_layer_sizes=(3,), max_iter=500, random_state=42),
}

# Hyperparameter optimization for LightGBM
space = {
    'num_leaves': hp.choice('num_leaves', range(10, 50)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', range(50, 200))
}

def objective(params):
    model = lgb.LGBMClassifier(**params, random_state=42)
    loocv = LOSOCV(model)
    _, test_metrics = loocv.run(features, targets)
    return -test_metrics['accuracy']  # Minimize negative accuracy

best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=Trials())

models["LightGBM (Optimized)"] = lgb.LGBMClassifier(**best_params, random_state=42)

# Run models and collect results
results = []
for name, model in models.items():
    loocv = LOSOCV(model)
    _, test_metrics = loocv.run(features, targets)
    results.append({"Model": name, "Accuracy": test_metrics["accuracy"]})

# Convert results to DataFrame and sort by accuracy
results_df = pd.DataFrame(results).sort_values(by="Accuracy")

# Print table
print(results_df.to_string(index=False))


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



[LightGBM] [Info] Number of positive: 14216, number of negative: 12012
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24225
[LightGBM] [Info] Number of data points in the train set: 26228, number of used features: 95
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.542016 -> initscore=0.168462
[LightGBM] [Info] Start training from score 0.168462
  5%|▌         | 1/20 [00:02<00:55,  2.94s/trial, best loss: -0.9731543624161074]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



...

100%|██████████| 20/20 [01:06<00:00,  3.33s/trial, best loss: -0.9731543624161074]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 14216, number of negative: 12012
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24225
[LightGBM] [Info] Number of data points in the train set: 26228, number of used features: 95
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.542016 -> initscore=0.168462
[LightGBM] [Info] Start training from score 0.168462
                          Model  Accuracy
                      KNN (k=3)  0.657718
                  Random Forest  0.936242
MLP (1 Hidden Layer, 3 Neurons)  0.949664
              SVM (Poly Kernel)  0.979866
           LightGBM (Optimized)  1.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [17]:
with open("model_results.txt", "w") as f:
    f.write(results_df.to_string(index=False))